# Sentiment Analysis on The Reddit Climate Change Dataset
by Santiago Segovia

### Analytical Process

1. Compare sentiment in data with Hugginface's approach
    * Use `distilbert-base-uncased` model to caluclate sentiment probability
    * Label data based on probabilities
    * Compare both sentiment metrics
2. Use data from Reddit to tune our own sentiment analysis model

### 1. Install Dependencies and Initial Setup

In [1]:
!pip install datasets transformers accelerate -q

In [2]:
import pandas as pd
import torch

from google.colab import drive
from transformers import pipeline

In [3]:
# Mount GDrive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Load data (takes ~1 min to load)
data_path = "/content/drive/Shareddrives/adv-ml-project/Data/"
comments = pd.read_csv(data_path + "comments_filtered.csv")
comments['date'] = pd.to_datetime(comments['date'])

<ipython-input-4-1d468a658f54>:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  comments = pd.read_csv(data_path + "comments_filtered.csv")


In [5]:
comments.head(2)

,subreddit.name,body,sentiment,date
0,askreddit,I think climate change tends to get some peopl...,0.6634,2022-08-31 23:56:46
1,askreddit,They need to change laws so it's more worth se...,0.469,2022-08-31 23:54:25


In [6]:
comments.dtypes

subreddit.name            object
body                      object
sentiment                 object
date              datetime64[ns]
dtype: object

### 2. Preprocess Data

In order to evaluate the performance of our model, we need to create a train-test split. We randomly pick 20% of the records and identify them as part of the testing dataset:

In [7]:
import random
random.seed(120938)

num_test_obs = round(comments.shape[0] * 0.2)
ids_test_obs = random.sample(range(comments.shape[0]), num_test_obs)

In [8]:
comments['test_split'] = 0
comments.loc[ids_test_obs,'test_split'] = 1

In [9]:
comments.head()

,subreddit.name,body,sentiment,date,test_split
0,askreddit,I think climate change tends to get some peopl...,0.6634,2022-08-31 23:56:46,0
1,askreddit,They need to change laws so it's more worth se...,0.469,2022-08-31 23:54:25,1
2,askreddit,That a big part of the solution to climate cha...,0.8937,2022-08-31 23:52:41,0
3,askreddit,&gt;Not climate change mind you\n\nHi. I have ...,0.0,2022-08-31 23:49:45,0
4,worldnews,"Climate change is not ""staring"" you in the fac...",-0.3453,2022-08-31 23:48:15,0


We need to convert our data to an iterable dataset to easily use Hugginface's functions. We use the `from_dict()` method from the `datasets` [module](https://huggingface.co/docs/datasets/en/create_dataset).

In [10]:
# Fill NaN values with empty strings, otherwise from_dict will raise an error
comments['body'] = comments['body'].fillna('')

In [11]:
from datasets import Dataset

# Create train and test data
train_data_dict = {"text": comments.loc[comments['test_split'] == 0, 'body'].tolist()}
test_data_dict = {"text": comments.loc[comments['test_split'] == 1, 'body'].tolist()}
train_data = Dataset.from_dict(train_data_dict)
test_data = Dataset.from_dict(test_data_dict)

In [12]:
# Example of structures
print(train_data[0])
print(test_data[0])

{'text': 'I think climate change tends to get some people riled up. \n\nWhen I was part of a debate club, they loved throwing that subject in. One case we had to discuss was whether or not it was okay to fly if it pollutes the air. A friend of mine on the team got very worked up because he loves to travel. At the end, we actually had to make up because our disagreement about flying got very heated.'}
{'text': "They need to change laws so it's more worth selling agriculture products in the US rather than export it.  They also need to change laws so there are monetary penalties for growing crops that are not particularly viable to an area's natural climate.  As it stands right now, my neighbor makes double the price per head of cattle by exporting out of country than he would selling right here.  All the people complaining about climate change on here should probably be complaining about this too."}


To [preprocess](https://huggingface.co/docs/transformers/preprocessing#everything-you-always-wanted-to-know-about-padding-and-truncation) our data, we will use [DistilBERT tokenizer](https://huggingface.co/docs/transformers/v4.15.0/en/model_doc/distilbert#transformers.DistilBertTokenizer):


In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Next, we prepare the text inputs for the model for both splits of our dataset (training and test) by using the [map method](https://huggingface.co/docs/datasets/en/process#map) with [batch processing](https://huggingface.co/docs/datasets/en/process#batch-processing):

In [14]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/729886 [00:00<?, ? examples/s]

Map:   0%|          | 0/182471 [00:00<?, ? examples/s]

Sentences aren’t always the same length which can be an issue because the model inputs need to have a uniform shape. Padding is a strategy for ensuring tensors are rectangular by adding a special padding token to shorter sentences. We use a [Data Collator](https://huggingface.co/docs/transformers/en/main_classes/data_collator) to convert our training samples to PyTorch tensors and concatenate them with the correct amount of [padding](https://huggingface.co/docs/transformers/preprocessing#pad):

In [15]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

### 3. Training the model

For training, we will be using the [Trainer API](https://huggingface.co/docs/transformers/v4.15.0/en/main_classes/trainer#transformers.Trainer), which is optimized for fine-tuning Transformers models such as DistilBERT.

First, we define DistilBERT as our base model:

In [17]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Even though the pre-trained model computes a loss, once we train the model it is just returning the logits. The following code ensures that a loss is returned during training:

In [30]:
from transformers import AutoModelForSequenceClassification

class CustomModel(AutoModelForSequenceClassification):
    def forward(self, input_ids=None, attention_mask=None, labels=None):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Retrieve the loss from the outputs
        loss = outputs.loss if labels is not None else None

        return {"loss": loss, "logits": outputs.logits}

# Create an instance of the custom model
model = CustomModel.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Then, we define the metrics we'll be using to evaluate how good is the fine-tuned model ([accuracy and f1 score](https://huggingface.co/metrics)):

In [18]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

Before training our model, we need to define the training arguments and define a Trainer with all the objects we constructed up to this point:

In [31]:
from transformers import TrainingArguments, Trainer

output_name = data_path + "Output"

training_args = TrainingArguments(
   output_dir=output_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


Now, we train the model on the dataset:

In [ ]:
trainer.train()